In [1]:
from models.retinaface import RetinaFace
from data import cfg_re50
import torch
import cv2
import numpy as np
from layers.functions.prior_box import PriorBox
import torch.backends.cudnn as cudnn
from torch.nn import DataParallel
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode, decode_landm

c:\Users\PC\.conda\envs\retinaface\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}

In [3]:
pretrained_weights = './weights/Resnet50/b8/lr0.001/optSGD/Resnet50_b8_lr1.0000000000000003e-05_optSGD_Final.pth'
model = RetinaFace(cfg=cfg_re50, phase='test')
device = torch.cuda.current_device()
pretrained_dict = torch.load(pretrained_weights, map_location=lambda storage, loc: storage.cuda(device))
pretrained_dict = remove_prefix(pretrained_dict, 'module.')
model.load_state_dict(pretrained_dict)

remove prefix 'module.'


<All keys matched successfully>

In [4]:
cudnn.benchmark = True
device = torch.device("cuda")
model = model.to(device)
model = DataParallel(model)

In [5]:
image_path = "./curve/test6.jpg"
img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
img = np.float32(img_raw)
im_height, im_width, _ = img.shape
scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
img -= (104, 117, 123)
img = img.transpose(2, 0, 1)
img = torch.from_numpy(img).unsqueeze(0)
img = img.to(device)
scale = scale.to(device)

# (loc, conf, landms), embedding = model(img)  # forward pass

In [6]:
# transform = transforms.Compose([
#     transforms.Resize((cfg_re50['image_size'], cfg_re50['image_size'])),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

In [7]:
# img = transform(img_raw)

In [8]:
(loc, conf, landms), embedding = model(img)  # forward pass

In [9]:
loc, loc.squeeze(0).shape, conf, conf.squeeze(0).shape, landms, landms.squeeze(0).shape

(tensor([[[ 0.5469,  1.5489, -0.8814,  0.2299],
          [ 0.9827,  1.7706, -1.4537, -0.1859],
          [-0.1422,  1.6518, -0.3592, -0.6741],
          ...,
          [-0.4425, -0.1530, -0.4372, -0.1865],
          [-0.9754, -0.5551, -0.7518,  0.7454],
          [-0.5474, -0.0828, -1.3114,  0.0774]]], device='cuda:0',
        grad_fn=<CatBackward>),
 torch.Size([24192, 4]),
 tensor([[[9.9936e-01, 6.3897e-04],
          [9.9952e-01, 4.8082e-04],
          [9.9890e-01, 1.0987e-03],
          ...,
          [9.9994e-01, 6.1348e-05],
          [9.9997e-01, 3.4573e-05],
          [9.9999e-01, 1.2661e-05]]], device='cuda:0',
        grad_fn=<SoftmaxBackward>),
 torch.Size([24192, 2]),
 tensor([[[-1.2888, -1.7257,  2.9366,  ...,  4.3994,  2.2572,  4.3963],
          [-0.7890, -1.0828,  2.5034,  ...,  3.7872,  2.2856,  3.5980],
          [-2.2388, -1.5540,  2.1273,  ...,  3.7382,  1.7620,  3.6749],
          ...,
          [-1.4398, -1.3708,  0.1921,  ...,  2.3055,  0.0543,  1.9738],
       

In [10]:
embedding

OrderedDict([('layer2',
              tensor([[[[0.0000e+00, 0.0000e+00, 8.1872e-01,  ..., 3.4432e-01,
                         0.0000e+00, 8.5564e-01],
                        [6.4227e-03, 0.0000e+00, 6.1718e-02,  ..., 0.0000e+00,
                         0.0000e+00, 5.9092e-01],
                        [1.2926e+00, 1.9650e-01, 4.1889e-01,  ..., 0.0000e+00,
                         5.9328e-01, 3.3760e-01],
                        ...,
                        [1.0651e+00, 1.3693e+00, 1.0829e+00,  ..., 3.9240e-01,
                         4.5326e-02, 3.8854e-02],
                        [1.0748e+00, 1.3686e+00, 2.0782e+00,  ..., 1.3366e+00,
                         9.5888e-01, 4.4663e-01],
                        [6.5448e-01, 1.6159e-01, 2.1342e-02,  ..., 0.0000e+00,
                         3.0699e-02, 6.6310e-01]],
              
                       [[2.1205e+00, 3.5062e+00, 2.2774e+00,  ..., 1.9434e+00,
                         2.2370e+00, 9.0633e-01],
                        [1.9

In [11]:
embedding['layer4'], embedding['layer4'].shape

(tensor([[[[0.0710, 0.5885, 1.5663,  ..., 2.0308, 2.1395, 2.4682],
           [0.1243, 0.0000, 0.3649,  ..., 0.0000, 0.3005, 0.4245],
           [0.7900, 0.7971, 1.0727,  ..., 0.0000, 0.0000, 0.3047],
           ...,
           [1.4366, 1.0539, 0.7332,  ..., 0.0365, 1.2494, 0.9550],
           [1.9746, 2.0293, 0.6127,  ..., 0.6417, 1.2643, 1.0407],
           [1.9142, 1.2008, 0.8996,  ..., 0.3239, 1.4140, 1.1496]],
 
          [[0.3497, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4381],
           [0.8152, 0.0000, 0.0000,  ..., 0.5053, 0.0868, 0.9682],
           [0.9059, 0.4991, 0.9231,  ..., 0.7791, 0.9768, 1.2625],
           ...,
           [0.0556, 0.1921, 0.8578,  ..., 0.6974, 0.0000, 1.6735],
           [0.0000, 0.0253, 0.7144,  ..., 0.0000, 0.8407, 2.1449],
           [0.9496, 0.7147, 0.9501,  ..., 0.9624, 1.6499, 2.0211]],
 
          [[0.0000, 0.7779, 0.1636,  ..., 0.0000, 0.0000, 0.0000],
           [0.2546, 0.4168, 0.6722,  ..., 0.5122, 0.6291, 0.0000],
           [0.2669, 0.45

In [12]:
embedding = embedding['layer4'].squeeze()  # Ambil embedding dari 'layer4' dan hilangkan dimensi yang tidak diperlukan
embedding_dim = embedding.numel()  # Hitung jumlah elemen di tensor
embedding = embedding.view(-1, embedding_dim)

In [13]:
embedding, embedding.shape

(tensor([[0.0710, 0.5885, 1.5663,  ..., 0.0000, 0.0000, 0.0815]],
        device='cuda:0', grad_fn=<ViewBackward>),
 torch.Size([1, 1179648]))

In [14]:
embedding.squeeze(0).shape

torch.Size([1179648])

In [15]:
arcface_input = embedding.view(512, -1)  # Reshape into a 512x(-1) tensor

In [16]:
arcface_input = arcface_input.mean(dim=1, keepdim=True)

In [17]:
arcface_input = arcface_input.t()

In [18]:
arcface_input, arcface_input.shape

(tensor([[0.6002, 0.5812, 0.5782, 0.5321, 0.5659, 0.5577, 0.5715, 0.5549, 0.6095,
          0.5565, 0.5690, 0.5544, 0.5931, 0.5938, 0.5638, 0.5528, 0.5873, 0.5882,
          0.5588, 0.5771, 0.5747, 0.5886, 0.5908, 0.5687, 0.5742, 0.5536, 0.5487,
          0.5708, 0.5772, 0.5658, 0.5599, 0.5731, 0.5335, 0.5819, 0.5713, 0.5600,
          0.5526, 0.5822, 0.5849, 0.5518, 0.6019, 0.5514, 0.5640, 0.5718, 0.5652,
          0.5933, 0.5902, 0.5630, 0.5729, 0.5863, 0.5416, 0.5859, 0.5479, 0.5807,
          0.5731, 0.5833, 0.5566, 0.5771, 0.5758, 0.5579, 0.5547, 0.5782, 0.5397,
          0.5894, 0.5672, 0.5398, 0.5815, 0.5902, 0.5727, 0.5823, 0.5642, 0.5882,
          0.5952, 0.5553, 0.5620, 0.5530, 0.5766, 0.5651, 0.5687, 0.5798, 0.5625,
          0.5879, 0.5936, 0.5882, 0.5555, 0.5601, 0.5734, 0.5665, 0.5410, 0.5877,
          0.5883, 0.5782, 0.5718, 0.5707, 0.5433, 0.5331, 0.5896, 0.5521, 0.5672,
          0.5503, 0.5938, 0.5852, 0.5691, 0.5887, 0.5613, 0.5809, 0.5960, 0.5737,
          0.5585